In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
n_value = np.array([1.0,3.0,5.0,8.0,10.0]).reshape(-1,1)
r_value = np.array([2,6,8]).reshape(-1,1)

LR_tune,N_value,R_value = np.meshgrid(lr_tune,n_value,r_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
N_value = N_value.flatten('F').reshape(-1,1)
R_value = R_value.flatten('F').reshape(-1,1)


lrnr_tune = np.hstack((LR_tune,N_value,R_value))

In [6]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [7]:
loss_thresh = 0.01

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [8]:
xt.shape

(500000, 2)

In [9]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    x_BC1 = np.random.uniform(size = N_I).reshape(-1,1)
    t_BC1 = np.zeros((N_I,1))
    samples = np.hstack((x_BC1,t_BC1))
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    
    x_BC2 = np.zeros((int(N_B/2),1))
    t_BC2 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC2,t_BC2))
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x_BC3 = np.ones((int(N_B/2),1))
    t_BC3 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC3,t_BC3))
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [10]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        self.omega1.requiresGrad = True
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.alpha.requiresGrad = True
        
        
        self.omega = Parameter((1/n_val)*torch.ones(rowdy_terms,len(layers)-2))
        self.omega.requiresGrad = True
        
        self.n = torch.tensor(n_val)

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred1 = self.forward(xt_test_tensor[:250000])
        y_pred1 = y_pred1.cpu().detach().numpy()
        
        y_pred2 = self.forward(xt_test_tensor[250000:])
          
        y_pred2 = y_pred2.cpu().detach().numpy()
        
        y_pred = np.vstack((y_pred1.reshape(-1,1),y_pred2.reshape(-1,1)))
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [13]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0

    xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_BC = torch.from_numpy(xt_BC).float().to(device)
    y_BC = torch.from_numpy(y_BC).float().to(device)

    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    nan_flag = 0
    for i in range(max_iter):
        train_step(xt_BC, y_BC, xt_coll,f_hat,i)

        loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
        
        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
            
        

    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))
    
    return nan_flag

In [14]:
nan_tune = []
for tune_reps in range(65,75):
#for tune_reps in range(1,5):
    max_reps = 10 #10
    max_iter = 100 #100
    label = "KG_rowdy_tune"+str(tune_reps)

    train_loss_full = []
    test_mse_full = []
    test_re_full = []
    alpha_full = []
    omega_full = []
    elapsed_time= np.zeros((max_reps,1))
    time_threshold = np.empty((max_reps,1))
    time_threshold[:] = np.nan
    epoch_threshold = max_iter*np.ones((max_reps,1))

    n_val = lrnr_tune[tune_reps,1]
    rowdy_terms = int(lrnr_tune[tune_reps,2])

    N_I = 200  #Total number of data points for 'y'
    N_B = 400
    N_f = 10000 #Total number of collocation points

    for reps in range(max_reps):
        print(reps)
        print(label)

        train_loss = []
        test_mse_loss = []
        test_re_loss = []
        alpha_val = []
        omega_val = []

        torch.manual_seed(reps*36)

        layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

        PINN = Sequentialmodel(layers,n_val,rowdy_terms)

        PINN.to(device)

        'Neural Network Summary'
        print(PINN)

        params = list(PINN.parameters())


        optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lrnr_tune[tune_reps,0], 
                                max_iter = 10, 
                                max_eval = 15, 
                                tolerance_grad = 1e-8, 
                                tolerance_change = 1e-8, 
                                history_size = 100, 
                                line_search_fn = 'strong_wolfe')




        nan_flag = train_model(max_iter,reps)
        
        torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
        train_loss_full.append(train_loss)
        test_mse_full.append(test_mse_loss)
        test_re_full.append(test_re_loss)
        #elapsed_time[reps] = time.time() - start_time
        alpha_full.append(alpha_val)
        omega_full.append(omega_val)
        
        if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break

        
      #print('Training time: %.2f' % (elapsed_time[reps]))

    mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full,  "omega": omega_full,"label": label}
    savemat(label+'.mat', mdic)

0
KG_rowdy_tune65
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 57.862 Test MSE 6.894908259881753 Test RE 1.2550825930192808
1 Train Loss 42.824253 Test MSE 8.039215796348868 Test RE 1.3552359188099874
2 Train Loss 32.18009 Test MSE 7.287693770666184 Test RE 1.2903369025662268
3 Train Loss 22.486507 Test MSE 6.621594680993979 Test 

91 Train Loss 0.0008516185 Test MSE 0.0010951439302209836 Test RE 0.01581771187676227
92 Train Loss 0.00081227254 Test MSE 0.0011171416430954954 Test RE 0.01597578402866897
93 Train Loss 0.0007796152 Test MSE 0.0011030297740105173 Test RE 0.015874559318712428
94 Train Loss 0.0007639015 Test MSE 0.001058870154374014 Test RE 0.015553545855694932
95 Train Loss 0.0007446669 Test MSE 0.000999271323203157 Test RE 0.015109488847425693
96 Train Loss 0.00071970746 Test MSE 0.0009316698276662054 Test RE 0.014589455234320805
97 Train Loss 0.00068437087 Test MSE 0.0008399172632559463 Test RE 0.013852441159385998
98 Train Loss 0.00065067434 Test MSE 0.0007683927349873053 Test RE 0.013249505973617437
99 Train Loss 0.00063657295 Test MSE 0.0007226968661254222 Test RE 0.012849497552894351
Training time: 149.71
1
KG_rowdy_tune65
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_feat

86 Train Loss 0.6073017 Test MSE 6.643394283748768 Test RE 1.231978346125666
87 Train Loss 0.6047297 Test MSE 6.660127359940273 Test RE 1.233528895594761
88 Train Loss 0.6023578 Test MSE 6.665850692269518 Test RE 1.234058793850415
89 Train Loss 0.6001961 Test MSE 6.676350088672029 Test RE 1.2350302958366617
90 Train Loss 0.597365 Test MSE 6.684918734676438 Test RE 1.2358225808391552
91 Train Loss 0.5952891 Test MSE 6.697333857706029 Test RE 1.236969623380605
92 Train Loss 0.59305143 Test MSE 6.689046485386205 Test RE 1.2362040649321955
93 Train Loss 0.59011483 Test MSE 6.703475263635418 Test RE 1.2375366394501268
94 Train Loss 0.58623403 Test MSE 6.722546418415997 Test RE 1.2392957635998545
95 Train Loss 0.5833111 Test MSE 6.727109632868244 Test RE 1.239716304615156
96 Train Loss 0.5805286 Test MSE 6.737121129617841 Test RE 1.2406384541042403
97 Train Loss 0.577999 Test MSE 6.758365210753535 Test RE 1.2425929595210952
98 Train Loss 0.5755487 Test MSE 6.756848721958776 Test RE 1.2424535

79 Train Loss 0.002041041 Test MSE 0.0016632230522044742 Test RE 0.019493207605490045
80 Train Loss 0.0019706613 Test MSE 0.001634569166187743 Test RE 0.019324564323498043
81 Train Loss 0.0018809326 Test MSE 0.0014679671505261746 Test RE 0.018313283935311654
82 Train Loss 0.0018019539 Test MSE 0.0013853057148149666 Test RE 0.017790201748270767
83 Train Loss 0.0017168404 Test MSE 0.0012884863044124778 Test RE 0.017157261226506182
84 Train Loss 0.0016533739 Test MSE 0.0012389372308613473 Test RE 0.01682413374921452
85 Train Loss 0.0015566065 Test MSE 0.0012079903118768737 Test RE 0.016612683301231022
86 Train Loss 0.0014808509 Test MSE 0.0012353078848642938 Test RE 0.016799473344980394
87 Train Loss 0.0014148942 Test MSE 0.0012076966263418842 Test RE 0.016610663748048444
88 Train Loss 0.0013666839 Test MSE 0.001120806523491761 Test RE 0.01600196754694429
89 Train Loss 0.001328354 Test MSE 0.0010745195316181518 Test RE 0.015668059687704877
90 Train Loss 0.001289484 Test MSE 0.001054327053

73 Train Loss 0.6227819 Test MSE 6.506211637389538 Test RE 1.219192137724493
74 Train Loss 0.61993665 Test MSE 6.5198190859104415 Test RE 1.220466414513799
75 Train Loss 0.6172387 Test MSE 6.515119675351409 Test RE 1.2200264861532066
76 Train Loss 0.61465967 Test MSE 6.525415675256085 Test RE 1.220990124149195
77 Train Loss 0.61217153 Test MSE 6.540483176185181 Test RE 1.222398974017285
78 Train Loss 0.61021346 Test MSE 6.54210199113158 Test RE 1.2225502407698658
79 Train Loss 0.6081921 Test MSE 6.563421801014651 Test RE 1.2245406819221132
80 Train Loss 0.6067282 Test MSE 6.56486705779096 Test RE 1.2246754956181718
81 Train Loss 0.6051221 Test MSE 6.576429662477358 Test RE 1.2257535227009517
82 Train Loss 0.60335934 Test MSE 6.588632776412457 Test RE 1.2268902395923966
83 Train Loss 0.6013475 Test MSE 6.584750600303086 Test RE 1.22652873015913
84 Train Loss 0.5996038 Test MSE 6.5883999547992795 Test RE 1.226868562174792
85 Train Loss 0.59762686 Test MSE 6.591885322296149 Test RE 1.2271

...

77 Train Loss 0.042666543 Test MSE 0.008908982354269705 Test RE 0.04511511925025722
78 Train Loss 0.041346114 Test MSE 0.008316189392821353 Test RE 0.0435883315823906
79 Train Loss 0.04055895 Test MSE 0.008163119749306273 Test RE 0.043185320195898706
80 Train Loss 0.039540783 Test MSE 0.00813641685128487 Test RE 0.04311462922009942
81 Train Loss 0.037504397 Test MSE 0.009632965042406022 Test RE 0.046912443287967086
82 Train Loss 0.036133684 Test MSE 0.008830821380400113 Test RE 0.044916779540702785
83 Train Loss 0.035370342 Test MSE 0.009163904346459096 Test RE 0.0457560297770072
84 Train Loss 0.0344426 Test MSE 0.007778637364473596 Test RE 0.042156041461854664
85 Train Loss 0.033547666 Test MSE 0.00702620042682236 Test RE 0.04006529339183675
86 Train Loss 0.03234738 Test MSE 0.0066790969189887235 Test RE 0.039063120472939784
87 Train Loss 0.030779723 Test MSE 0.006234195392101658 Test RE 0.03773968449985749
88 Train Loss 0.030016175 Test MSE 0.00679827065134237 Test RE 0.0394100771995

72 Train Loss 0.6966216 Test MSE 3.470189522250998 Test RE 0.8903989872615964
73 Train Loss 0.674355 Test MSE 3.4076643508007236 Test RE 0.8823410112903991
74 Train Loss 0.66806155 Test MSE 3.436433110401878 Test RE 0.8860577075118891
75 Train Loss 0.6518774 Test MSE 3.4472647713689724 Test RE 0.8874530388691894
76 Train Loss 0.64084965 Test MSE 3.4465788388239496 Test RE 0.8873647423143609
77 Train Loss 0.6307227 Test MSE 3.4423656025830423 Test RE 0.8868222013277353
78 Train Loss 0.6227456 Test MSE 3.4465054137952147 Test RE 0.8873552901689489
79 Train Loss 0.61226857 Test MSE 3.462035233962372 Test RE 0.8893522374735415
80 Train Loss 0.5969833 Test MSE 3.477454889102875 Test RE 0.8913305924071455
81 Train Loss 0.591748 Test MSE 3.4820495590323723 Test RE 0.8919192438947265
82 Train Loss 0.58516514 Test MSE 3.4657798718792012 Test RE 0.8898330820972483
83 Train Loss 0.56960815 Test MSE 3.475544217544357 Test RE 0.8910856900198207
84 Train Loss 0.561797 Test MSE 3.49140932693015 Test 

68 Train Loss 1.1763097 Test MSE 5.810519908271392 Test RE 1.1521673313488143
69 Train Loss 1.1714114 Test MSE 5.816601063483845 Test RE 1.1527700894694008
70 Train Loss 1.1557509 Test MSE 5.83771487189176 Test RE 1.1548604270254963
71 Train Loss 1.1468188 Test MSE 5.856603974828222 Test RE 1.1567273099009108
72 Train Loss 1.1293781 Test MSE 5.876712404046046 Test RE 1.1587113981182753
73 Train Loss 1.1235497 Test MSE 5.871990768700255 Test RE 1.1582458221569623
74 Train Loss 1.1087127 Test MSE 5.8645352329863485 Test RE 1.1575102891665372
75 Train Loss 1.102303 Test MSE 5.859763655516978 Test RE 1.1570392992493506
76 Train Loss 1.0995529 Test MSE 5.86617627603368 Test RE 1.1576722279453007
77 Train Loss 1.0872561 Test MSE 5.864193149683229 Test RE 1.1574765293968547
78 Train Loss 1.0839986 Test MSE 5.862472794494774 Test RE 1.1573067347834896
79 Train Loss 1.0788964 Test MSE 5.845536486475881 Test RE 1.1556338332064022
80 Train Loss 1.0613058 Test MSE 5.872232783084567 Test RE 1.15826

64 Train Loss 0.07309313 Test MSE 0.02774754763926722 Test RE 0.07961966814841277
65 Train Loss 0.068059415 Test MSE 0.02931904644172884 Test RE 0.08184327149932134
66 Train Loss 0.06281075 Test MSE 0.026931400937017003 Test RE 0.07843999060896714
67 Train Loss 0.059635356 Test MSE 0.025098287479545525 Test RE 0.07572339988674427
68 Train Loss 0.05571679 Test MSE 0.021989406190857118 Test RE 0.0708785478369065
69 Train Loss 0.05231266 Test MSE 0.02581363547776196 Test RE 0.07679494731043471
70 Train Loss 0.04979058 Test MSE 0.023190504866323937 Test RE 0.0727885657580735
71 Train Loss 0.04709091 Test MSE 0.020131325338630036 Test RE 0.06781788538440607
72 Train Loss 0.045357376 Test MSE 0.018986540433202817 Test RE 0.06586140323796157
73 Train Loss 0.04274804 Test MSE 0.017589866758532993 Test RE 0.06339271250349113
74 Train Loss 0.040333256 Test MSE 0.01723823563662542 Test RE 0.06275588619549119
75 Train Loss 0.036836207 Test MSE 0.014114470143345789 Test RE 0.056785878643273106
76 T

48 Train Loss 1.2343979 Test MSE 2.8910085705826867 Test RE 0.812704659783897
49 Train Loss 1.2235821 Test MSE 2.916347889277717 Test RE 0.8162585154658708
50 Train Loss 1.1941029 Test MSE 2.879983878173672 Test RE 0.811153578744683
51 Train Loss 1.1554756 Test MSE 2.80964986720392 Test RE 0.8011874937168122
52 Train Loss 1.1306058 Test MSE 2.8002837918014785 Test RE 0.7998509843684803
53 Train Loss 1.0942945 Test MSE 2.8092646570859414 Test RE 0.8011325694161645
54 Train Loss 1.0716008 Test MSE 2.817700229060073 Test RE 0.8023344757316354
55 Train Loss 1.0444847 Test MSE 2.8148717185218386 Test RE 0.8019316682732704
56 Train Loss 1.0122734 Test MSE 2.8458546407470697 Test RE 0.8063329684051423
57 Train Loss 1.0003369 Test MSE 2.8476217940020447 Test RE 0.8065832785888399
58 Train Loss 0.97166336 Test MSE 2.8203670630365827 Test RE 0.8027140737413203
59 Train Loss 0.9595258 Test MSE 2.8060783224794164 Test RE 0.8006781087335048
60 Train Loss 0.9510659 Test MSE 2.8075029004957015 Test R

In [15]:
import scipy.io as sio
import numpy as np

In [16]:
for tune_reps in range(75):
    label = "KG_rowdy_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

0   0.572127943642081
1   0.5853083824029831
2   0.606240757897077
3   0.6209681852381319
4   0.556137928468764
5   0.4381393935511763
6   0.630783227425957
7   0.5573664283666544
8   nan
9   0.6408324093289102
10   0.49892525904487084
11   0.46167696233447675
12   0.6792925670333688
13   0.6266449039236259
14   nan
15   [[1.4002289  1.40221893 1.49637238        nan]]
16   [[nan]]
17   0.6419719932458458
18   0.5328653725509389
19   0.5692298633470927
20   [[1.40337925 1.35857195        nan]]
21   0.6034983793563642
22   [[nan]]
23   0.6407499706994069
24   0.5659237130434658
25   0.7309730522732151
26   nan
27   [[1.19587968        nan]]
28   0.8004759224384494
29   0.8328352104010938
30   0.7578483115541701
31   [[nan]]
32   [[nan]]
33   0.7379399512771065
34   0.7842682093828542
35   0.7411595277220534
36   0.7189297113491893
37   0.7589339070131473
38   [[nan]]
39   [[nan]]
40   [[nan]]
41   [[nan]]
42   [[nan]]
43   [[nan]]
44   nan
45   0.7193360238488018
46   [[nan]]
47   0.8099

In [18]:
lrnr_tune[5]

array([0.1, 1. , 2. ])